# Understanding Large Language Models: A Step-by-Step Journey

## Our Goal
In this notebook, we will understand how language models predict text by following the complete process of predicting "we love deep learning" word by word. We'll explore four fundamental concepts that make modern language models like ChatGPT work:

1. **Forward Pass**: How models generate predictions
2. **Loss Calculation**: How we measure prediction quality  
3. **Backpropagation**: How we identify what needs improvement
4. **Gradient Descent**: How we make those improvements

## Our Vocabulary and Target
We'll work with a simple vocabulary to keep things clear and manageable. Our model will learn to predict each word in our target sequence step by step.

In [1]:
# Setup our simple vocabulary and target sequence
VOCAB = ["<BOS>", "we", "love", "deep", "learning", "<EOS>", "the", "is", "great", "model", "hello", "world"]
target_sequence = ["we", "love", "deep", "learning"]

print("VOCABULARY:", VOCAB)
print("TARGET SEQUENCE:", target_sequence)
print("VOCABULARY SIZE:", len(VOCAB))

# Initialize simple model parameters (weights) - these will be updated during training
# In real models, these would be millions or billions of parameters
model_parameters = {
    'layer1_weights': [0.1, -0.3, 0.5, 0.2, -0.1, 0.4, 0.8, -0.2, 0.3, -0.5, 0.7, -0.4],
    'layer2_weights': [0.2, 0.1, -0.4, 0.6, 0.3, -0.2, -0.1, 0.5, -0.3, 0.4, -0.6, 0.1],
    'output_weights': [0.3, -0.1, 0.4, -0.2, 0.5, 0.1, -0.3, 0.2, 0.6, -0.4, 0.1, 0.3]
}

print("\nInitial model parameters (simplified representation):")
print("Layer 1 weights:", len(model_parameters['layer1_weights']), "parameters")
print("Layer 2 weights:", len(model_parameters['layer2_weights']), "parameters") 
print("Output weights:", len(model_parameters['output_weights']), "parameters")

VOCABULARY: ['<BOS>', 'we', 'love', 'deep', 'learning', '<EOS>', 'the', 'is', 'great', 'model', 'hello', 'world']
TARGET SEQUENCE: ['we', 'love', 'deep', 'learning']
VOCABULARY SIZE: 12

Initial model parameters (simplified representation):
Layer 1 weights: 12 parameters
Layer 2 weights: 12 parameters
Output weights: 12 parameters


## What Are Logits?

Logits are the raw numerical scores that a language model assigns to every word in its vocabulary when predicting the next word. Think of logits as the model's initial "gut feeling" about how likely each word is to come next, before any normalization.

**Key Properties of Logits:**
- They can be any real number (positive, negative, large, small)
- Higher logits indicate the model thinks a word is more likely
- Lower logits indicate the model thinks a word is less likely
- They are computed by passing the current context through the neural network layers

Let's see what logits look like when our model tries to predict the first word after the beginning-of-sequence token.

In [2]:
def demonstrate_logits():
    # Pre-calculated realistic logit values for predicting first word after <BOS>
    logits_after_bos = {
        "we": 2.1,      # Target word - decent score
        "the": 3.2,     # Highest - most common starter
        "hello": 2.8,   # High - common greeting
        "is": 1.5,      # Medium - possible starter
        "love": -0.5,   # Low - uncommon starter
        "deep": -1.2,   # Lower - rare starter
        "learning": -2.0, # Very low - very rare starter
        "<EOS>": -10.0, # Impossible - can't start with end token
    }
    
    print("Logit scores for predicting first word after <BOS>:")
    print("=" * 50)
    for word, logit in sorted(logits_after_bos.items(), key=lambda x: x[1], reverse=True):
        marker = " <- Our target!" if word == "we" else ""
        print(f"{word:10}: {logit:6.1f}{marker}")
    
    print(f"\nNotice how 'the' has the highest logit ({logits_after_bos['the']}) because it's")
    print("the most common way to start sentences in English.")
    print(f"Our target word 'we' has a logit of {logits_after_bos['we']}, which is decent")
    print("but not the highest - the model will need training to improve this!")
    
    return logits_after_bos

# Run the demonstration
logits_step1 = demonstrate_logits()

Logit scores for predicting first word after <BOS>:
the       :    3.2
hello     :    2.8
we        :    2.1 <- Our target!
is        :    1.5
love      :   -0.5
deep      :   -1.2
learning  :   -2.0
<EOS>     :  -10.0

Notice how 'the' has the highest logit (3.2) because it's
the most common way to start sentences in English.
Our target word 'we' has a logit of 2.1, which is decent
but not the highest - the model will need training to improve this!
